<a href="https://colab.research.google.com/github/andresbadillo/ConversionMatrizHoraria/blob/master/archivos_inhemeter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cargar archivos de lectura de los medidores Inhemeter

In [ ]:
import pandas as pd
from google.colab import files

# Paso 1: Cargar el archivo de texto
uploaded = files.upload()
input_file_path = list(uploaded.keys())[0]
output_file_path = input_file_path.replace('.txt', '.csv')

# Leer el archivo
with open(input_file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Eliminar las primeras 6 filas
lines = lines[6:]

# Identificar la línea donde comienzan los datos
data_start_index = None
for i, line in enumerate(lines):
    if line.startswith("No.|Date Time"):
        data_start_index = i
        break

# Extraer los datos y la cabecera
header = lines[data_start_index].strip().split('|')
data_lines = lines[data_start_index + 1:]

# Crear una lista de diccionarios para almacenar los datos
data = []
for line in data_lines:
    values = line.strip().split('|')
    if len(values) == len(header):
        data.append(dict(zip(header, values)))

# Crear un DataFrame
df = pd.DataFrame(data)

# Convertir las columnas numéricas y de fecha al tipo correcto
for col in df.columns:
    try:
        df[col] = pd.to_numeric(df[col])
    except ValueError:
        try:
            df[col] = pd.to_datetime(df[col])
        except ValueError:
            pass

# Eliminar filas donde "Date Time" tenga datos de 30 minutos
df = df[~df['Date Time'].dt.minute.isin([30])]

# Eliminar la columna "No."
df = df.drop(columns=['No.'])

# Calcular la energía consumida por hora para cada energía
df['kWhR'] = df['Positive active energy (A+) total(kWh)'].diff().shift(-1)
df['kWhD'] = df['Negative active energy (A-) total(kWh)'].diff().shift(-1)
df['kVarR'] = df['Positive reactive energy (Q+) total(kvarh)'].diff().shift(-1)
df['kVarD'] = df['Negative reactive energy (Q-) total(kvarh)'].diff().shift(-1)

# Exportar a CSV con delimitador ";"
df.to_csv(output_file_path, index=False, encoding='utf-8', sep=';')

print(f"El archivo CSV ha sido creado: {output_file_path}")


## Exportar la matriz horaria

In [ ]:
# Eliminar columnas innecesarias
df = df.drop(columns=[
    'Positive active energy (A+) total(kWh)',
    'Negative active energy (A-) total(kWh)',
    'Positive reactive energy (Q+) total(kvarh)',
    'Negative reactive energy (Q-) total(kvarh)'
])

# Separar la columna "Date Time" en "Date" y "Time"
df['Date'] = df['Date Time'].dt.date
df['Time'] = df['Date Time'].dt.hour
df = df.drop(columns=['Date Time'])

# Reordenar columnas para que "Date" y "Time" estén al principio
columns_order = ['Date', 'Time'] + [col for col in df.columns if col not in ['Date', 'Time']]
df = df[columns_order]

# Exportar provisionalmente a un archivo CSV
provisional_output_path = 'provisional.csv'
df.to_csv(provisional_output_path, index=False, encoding='utf-8', sep=';')

print(f"El archivo provisional CSV ha sido creado: {provisional_output_path}")

# Crear una matriz horaria con una columna independiente para cada tipo de energía
df_melted = df.melt(id_vars=['Date', 'Time'], value_vars=['kWhR', 'kWhD', 'kVarR', 'kVarD'],
                    var_name='Energy Type', value_name='Value')

# Pivotar la columna "Time" para obtener una matriz horaria
df_pivot = df_melted.pivot_table(
    index=['Date', 'Energy Type'],
    columns='Time',
    values='Value',
    aggfunc='sum'
)

# Aplanar el índice de columnas para un formato tipo matríz
df_pivot.columns = [f"Hour{col}" for col in df_pivot.columns]
df_pivot.reset_index(inplace=True)

# Exportar la matriz horaria a un archivo CSV
matrix_output_path = 'hourly_matrix.csv'
df_pivot.to_csv(matrix_output_path, index=False, encoding='utf-8', sep=';')

print(f"El archivo matriz horaria CSV ha sido creado: {matrix_output_path}")

## Transformar otro tipo de archivo en matriz horaria

In [ ]:
import pandas as pd
from google.colab import files

# Paso 1: Cargar el archivo de Excel
uploaded = files.upload()
input_file_path = list(uploaded.keys())[0]
output_file_path = input_file_path.replace('.xlsx', '_processed.csv')

# Leer el archivo Excel
df = pd.read_excel(input_file_path)

# Eliminar la columna No.
df = df.drop(columns=['No.'], errors='ignore')

# Renombrar las columnas
df = df.rename(columns={
    'Positive active energy (A+) total(kWh)': 'kWhR',
    'Negative active energy (A-) total(kWh)': 'kWhD',
    'Positive reactive energy (Q+) total(kvarh)': 'kVarR',
    'Negative reactive energy (Q-) total(kvarh)': 'kVarD'
})

# Convertir la columna de fecha y hora al formato datetime
df['Date Time'] = pd.to_datetime(df['Date Time'])

# Extraer la fecha, la hora y los minutos
df['Date'] = df['Date Time'].dt.date
df['Hour'] = df['Date Time'].dt.hour
df['Minute'] = df['Date Time'].dt.minute

# Ajustar la hora para la agrupación correcta
df['Adjusted Hour'] = df['Hour']
df.loc[df['Minute'] == 0, 'Adjusted Hour'] = df['Hour'] - 1

df_grouped = df[df['Minute'].isin([15, 30, 45]) | ((df['Minute'] == 0) & (df['Adjusted Hour'] >= 0))]
df_grouped = df_grouped.drop(columns=['Date Time', 'Hour', 'Minute']).groupby(['Date', 'Adjusted Hour']).sum().reset_index()
df_grouped = df_grouped.rename(columns={'Adjusted Hour': 'Hour'})

# Reordenar columnas
columns_order = ['Date', 'Hour'] + [col for col in df_grouped.columns if col not in ['Date', 'Hour', 'Date Time']]
df_grouped = df_grouped[columns_order]

# Crear una matriz horaria con una columna independiente para cada tipo de energía
df_melted = df_grouped.melt(id_vars=['Date', 'Hour'], value_vars=['kWhR', 'kWhD', 'kVarR', 'kVarD'],
                            var_name='Energy Type', value_name='Value')

# Pivotar la columna "Hour" para obtener una matriz horaria
df_pivot = df_melted.pivot_table(
    index=['Date', 'Energy Type'],
    columns='Hour',
    values='Value',
    aggfunc='sum'
)

# Aplanar el índice de columnas para un formato tipo matriz
df_pivot.columns = [f"Hour{col}" for col in df_pivot.columns]
df_pivot.reset_index(inplace=True)

# Exportar la matriz horaria a un archivo CSV
matrix_output_path = input_file_path.replace('.xlsx', '_hourly_matrix.csv')
df_pivot.to_csv(matrix_output_path, index=False, encoding='utf-8', sep=';')

print(f"El archivo matriz horaria CSV ha sido creado: {matrix_output_path}")